<a href="https://colab.research.google.com/github/ykato27/decision-transformer/blob/master/Decision_Transformer_OpenAIGym_atari_Pong_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### decision-transformerのリポジトリをクローン

In [1]:
!git clone https://github.com/kzl/decision-transformer.git
%cd decision-transformer/atari

Cloning into 'decision-transformer'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 135 (delta 63), reused 122 (delta 53), pack-reused 0
Receiving objects: 100% (135/135), 249.31 KiB | 8.90 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/decision-transformer/atari


### 実行環境の確認

In [2]:
!uname -m && cat /etc/*release | head -n 2

x86_64
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=18.04


In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [4]:
%%bash

apt-get install build-essential -qq
gcc --version

gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [5]:
%%bash

uname -r
ls -d /usr/src/linux-headers-*

5.4.104+
/usr/src/linux-headers-4.15.0-163
/usr/src/linux-headers-4.15.0-163-generic


### CUDA Toolkitのインストール

In [6]:
!cat /proc/driver/nvidia/version

cat: /proc/driver/nvidia/version: No such file or directory


In [7]:
%%bash

apt update -qq;
rm ./cuda-repo-ubuntu1804_*amd64.deb
wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb

yes | dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb > /dev/null 2>&1
apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub > /dev/null 2>&1
apt-get update -qq
apt-get install cuda -qq

79 packages can be upgraded. Run 'apt list --upgradable' to see them.
Selecting previously unselected package cuda-toolkit-config-common.
(Reading database ... 155225 files and directories currently installed.)
Preparing to unpack .../00-cuda-toolkit-config-common_11.5.117-1_all.deb ...
Unpacking cuda-toolkit-config-common (11.5.117-1) ...
Selecting previously unselected package cuda-toolkit-11-config-common.
Preparing to unpack .../01-cuda-toolkit-11-config-common_11.5.117-1_all.deb ...
Unpacking cuda-toolkit-11-config-common (11.5.117-1) ...
Selecting previously unselected package cuda-toolkit-11-5-config-common.
Preparing to unpack .../02-cuda-toolkit-11-5-config-common_11.5.117-1_all.deb ...
Unpacking cuda-toolkit-11-5-config-common (11.5.117-1) ...
Selecting previously unselected package cuda-cudart-11-5.
Preparing to unpack .../03-cuda-cudart-11-5_11.5.117-1_amd64.deb ...
Unpacking cuda-cudart-11-5 (11.5.117-1) ...
Selecting previously unselected package cuda-nvrtc-11-5.
Preparin



rm: cannot remove './cuda-repo-ubuntu1804_*amd64.deb': No such file or directory
--2021-12-18 07:04:48--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2940 (2.9K) [application/x-deb]
Saving to: ‘cuda-repo-ubuntu1804_10.0.130-1_amd64.deb’

     0K ..                                                    100%  118M=0s

2021-12-18 07:04:48 (118 MB/s) - ‘cuda-repo-ubuntu1804_10.0.130-1_amd64.deb’ saved [2940/2940]



In [8]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Thu_Nov_18_09:45:30_PST_2021
Cuda compilation tools, release 11.5, V11.5.119
Build cuda_11.5.r11.5/compiler.30672275_0


In [9]:
!ls -d /usr/local/cuda-10.?
!which nvcc

/usr/local/cuda-10.0  /usr/local/cuda-10.1
/usr/local/cuda/bin/nvcc


In [10]:
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-10.0/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-10.0/lib64:{ld}"

In [11]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [12]:
!pip install psutil \
    opencv-python \
    atari-py \
    pyprind \
    tensorflow-gpu \
    absl-py \
    atari-py \
    gin-config \
    gym \
    blosc
!pip install git+https://github.com/google/dopamine.git

     |████████████████████████████████| 489.6 MB 23 kB/s 
     |████████████████████████████████| 2.6 MB 24.7 MB/s 
  Cloning https://github.com/google/dopamine.git to /tmp/pip-req-build-2gmmvnwh
  Running command git clone -q https://github.com/google/dopamine.git /tmp/pip-req-build-2gmmvnwh
     |████████████████████████████████| 207 kB 5.1 MB/s 
     |████████████████████████████████| 18.3 MB 46 kB/s 
     |████████████████████████████████| 352 kB 56.2 MB/s 
     |████████████████████████████████| 126 kB 52.8 MB/s 
     |████████████████████████████████| 65 kB 3.3 MB/s 
  Created wheel for dopamine-rl: filename=dopamine_rl-4.0.2-py3-none-any.whl size=2795130 sha256=3b5cc01e6c5d63e0d508523fb23df00bebac6b392fa86ec84fac1589993d7e71
  Stored in directory: /tmp/pip-ephem-wheel-cache-q275xqwl/wheels/1c/f5/fe/97ec8a23ae6d332c2f007a5fe95bb68ac43144656b4df7067b
Successfully built dopamine-rl
  Attempting uninstall: dopamine-rl
    Found existing installation: dopamine-rl 1.0.5
    Uninstalli

### Downloading datasets

`mkdir [DIRECTORY_NAME]` \
`gsutil -m cp -R gs://atari-replay-datasets/dqn/[GAME_NAME] [DIRECTORY_NAME]`

In [13]:
%mkdir atari-game
!gsutil -m cp -R gs://atari-replay-datasets/dqn/Pong atari-game

Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.0.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.10.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.11.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.1.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.12.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.13.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.17.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.14.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.15.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.16.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1/replay_logs/$store$_action_ckpt.18.gz...
Copying gs://atari-replay-datasets/dqn/Pong/1

In [14]:
!python run_dt_atari.py \
    --seed 10 \
    --context_length 50 \
    --epochs 5 \
    --model_type 'reward_conditioned' \
    --num_steps 500000 \
    --num_buffers 50 \
    --game 'Pong' \
    --batch_size 512 \
    --data_dir_prefix atari-game/

loading from buffer 9 which has 0 already loaded
tcmalloc: large alloc 7056007168 bytes == 0x5621e2dc2000 @  0x7f8c60fc61e7 0x7f8c5e1bb46e 0x7f8c5e20bc7b 0x7f8c5e1bece8 0x5621aeab3045 0x5621aea73d49 0x5621aeae794f 0x5621aeae19ee 0x5621aea74bda 0x5621aeae3737 0x5621aeae19ee 0x5621aea74bda 0x5621aeae3737 0x5621aea74afa 0x5621aeae2c0d 0x5621aea74afa 0x5621aeae2c0d 0x5621aea74afa 0x5621aeae2c0d 0x5621aeae19ee 0x5621aea7548c 0x5621aeab6159 0x5621aeab30a4 0x5621aea73d49 0x5621aeae794f 0x5621aea74afa 0x5621aeae2915 0x5621aeae19ee 0x5621aeae16f3 0x5621aebab4c2 0x5621aebab83d
this buffer has 23485 loaded transitions and there are now 23485 transitions total divided into 10 trajectories
loading from buffer 36 which has 0 already loaded
tcmalloc: large alloc 7056007168 bytes == 0x5623b6994000 @  0x7f8c60fc61e7 0x7f8c5e1bb46e 0x7f8c5e20bc7b 0x7f8c5e1bece8 0x5621aeab3045 0x5621aea73d49 0x5621aeae794f 0x5621aeae19ee 0x5621aea74bda 0x5621aeae3737 0x5621aeae19ee 0x5621aea74bda 0x5621aeae3737 0x5621aea